## 1 Acessar site e coletar os PDFs

In [2]:
## instalar dependencias
%env
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
### Imports, variaveis globais 

# import requests
# from bs4 import BeautifulSoup

# import re #regex
# from datetime import datetime as dt
# import pandas as pd

URL_CEASA = 'https://www.ceasa.rj.gov.br'
URL_CEASA_cotacao = URL_CEASA+'Cota%C3%A7%C3%A3o'

PASTA_PDFs = "./pdfs/"
PASTA_DADOS = "./dados/"

ceasa_lista_pdf = PASTA_DADOS+"ceasa_lista_pdf.csv"

##### auxs

In [48]:
# barra de progresso
# font:https://stackoverflow.com/questions/6169217/replace-console-output-in-python

def print_r(str:str):
    print(str, end='\r')


def print_percent_done(index, total, progress_state=None, bar_len=25, title='Processando'):
    '''
    index is expected to be 0 based index. 
    0 <= index < total
    '''
    percent_done = (index+1)/total*100
    percent_done = round(percent_done, 1)

    done = round(percent_done/(100/bar_len))
    togo = bar_len-done

    done_str = '█'*int(done)
    togo_str = '░'*int(togo)

    progress_state = f"\t Stage: {progress_state}" if progress_state else ''

    print(f'\t⏳{title}: [{done_str}{togo_str}] {percent_done}% done {progress_state}', end='\r')

    if round(percent_done) == 100:
        print('\t✅')


### Coleta linear

In [ ]:
%%script echo skipping

import requests
from bs4 import BeautifulSoup

import re #regex
from datetime import datetime as dt
import pandas as pd


def foi_extraida(url):
	return True if url in((df_extração['URL'].eq(url))) else False

def pegar_links(url):
	"""
		Função recursiva que entra no site em busca links das tags 'a' na div com id="main" e entra no novo site em busta de links. ele para quando encrontra uma url com '.pdf'.

		Args:
			url (str):endereço da pagina html a ser percorrida
				(opcional)

		Return:
			None
	"""
	
	reqs = requests.get(url)
	soup = BeautifulSoup(reqs.text, 'html.parser')

	for link in soup.select('#main a'):
		
		a = str(link.get('href'))
		print_r(f'Qtds pdfs encontrados {len(urls)}, link atual: {a}')
		if ".pdf" in a:
			a = 'https://www.ceasa.rj.gov.br'+ a
			if not foi_extraida(a):
				continue
			urls.append(a)

		else:
				pegar_links(a)

def extrair_dados_url(url):
	"""
			Pega a url do PDF e extrai data, nome do documento e link do pdf

		Args:
			link: str
				url do documento pdf

		Return:
			url: str
			nome_arquivo: str
			data: str
	"""
	# pega nome do arquivo na URL
	nome_arquivo = url.split("/")[-1]
	nome_arquivo = requests.utils.unquote(nome_arquivo) # type: ignore

	# padrao de dd mm yyyy para data
	matches = re.findall(r'(\d{2})\s(\d{2})\s(\d{4})', nome_arquivo)
	data = dt.strptime("/".join(matches[0]), "%d/%m/%Y")

	return url,"CEASA-RJ_" + nome_arquivo.replace(" ", "_") ,data.strftime("%d-%m-%Y")

def pegar_pdf(url,nome_arquivo):
	# conferir se ja está na base local
	response = requests.get(url)

	if response.status_code == 200:
		with open(PASTA_PDFs+nome_arquivo, "wb") as f:
			f.write(response.content)
		
		return True

def add_csv(url, nome_arquivo,data):
	data_to_file = {
		'URL': [url],
		'nome_arquivo': [nome_arquivo],
		'data': [data]
	} 
	df = pd.DataFrame(data_to_file)
	df.to_csv(ceasa_lista_pdf, mode='a', index=False, header=False, sep=";")

def atualizar_base_pdf():
	pegar_links(URL_CEASA_cotacao)
	
	for x in urls[:]:
		print_percent_done(
		len(df_extração) - len(urls), 
		len(df_extração), 
		str(f'pdf extraidos {len(urls)} de {len(df_extração)} \n\t link atual {x}'))

		# if foi_extraida:
		# 	continue
		
		url, nome_arquivo, data = extrair_dados_url(x)
		pegar_pdf(url,nome_arquivo)
		add_csv(url,nome_arquivo,data)


# init

try:
    urls # type: ignore
except:
	urls = [] 


try:
	df_extração = pd.read_csv(ceasa_lista_pdf,sep=";")
except:
	df_extração = pd.read_csv(ceasa_lista_pdf,sep=";", names = ["URL", "nome_arquivo","data"])
	df_extração.to_csv(ceasa_lista_pdf, sep=';', index=False)

atualizar_base_pdf()


## Tratando os PDF

##### Compressão paralelizada (muito custoso)

In [ ]:
%%script echo skipping

import PyPDF2
import concurrent.futures

def compress_pdf(input_path, output_path):
    with open(input_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        pdf_writer = PyPDF2.PdfWriter()

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            page.compress_content_streams()
            pdf_writer.add_page(page)

        with open(output_path, 'wb') as output_file:
            pdf_writer.write(output_file)

def process_pdf(file_name):
    print(f'arquivo atual: {file_name}')
    compress_pdf(PASTA_PDFs + file_name, PASTA_PDFs + file_name)

# Sua lista de arquivos
files_to_process = df_extração["nome_arquivo"].tolist()

# Número máximo de threads (ajuste conforme necessário)
max_threads = 4

# Usando ThreadPoolExecutor para paralelizar
with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
    futures = [executor.submit(process_pdf, file_name) for file_name in files_to_process]

    # Esperar que todas as threads concluam
    for future in concurrent.futures.as_completed(futures):
        count += 1 # type: ignore
        print_percent_done(count, len(files_to_process), future.result()) # type: ignore


### Lendo Tabelas 

Apartir de 01-03-2023 ouver mudança na formatação da tabela.
trataremos as duas versões de PDF:

In [9]:
import pandas as pd
import pdfplumber


pdf_modelo_v1 = PASTA_PDFs + "CEASA-RJ_Boletim_diário_de_preços__03_01_2022_0.pdf" 
pdf_modelo_v2 = PASTA_PDFs + "CEASA-RJ_Boletim_diário_de_preços__24_04_2023.pdf" 

def extrair_tabelas(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        # Itera sobre todas as páginas do PDF
        for page_number in range(len(pdf.pages)):
            page = pdf.pages[page_number]

            # Extrai todas as tabelas na página
            tables = page.extract_tables()

            # Itera sobre todas as tabelas na página
            for table_number, table in enumerate(tables):
                print(f"Tabela {table_number + 1} na página {page_number + 1}:")
                
                # Itera sobre todas as linhas da tabela
                for row_number, row in enumerate(table):
                    print(f"  Linha {row_number + 1}: {row}")

                print("\n" + "="*50 + "\n")  # Separador entre tabelas


def extrair_tabela_pdf(pdf_path, pagina=0):
    # Abre o PDF com pdfplumber
    with pdfplumber.open(pdf_path) as pdf:
        # Seleciona a página desejada
        page = pdf.pages[pagina]
        
        # Extrai a tabela como uma lista de dicionários
        table_data = page.extract_table()

        # Converte a lista de dicionários para um DataFrame do pandas
        df = pd.DataFrame(table_data)

    return df

t = extrair_tabela_pdf(PASTA_PDFs+'CEASA-RJ_Boletim_diário_de_preços__09_11_2023.pdf')
t.head()

,0,1,2,3,4,5,6
0,None,Dia Semana:,quinta-feira,None,None,,09/11/2023
1,PRODUTOS,TIPO,UNIDADE EMBALAGEM,VARIAÇÃO\nULTIMOS\n12 MESES,MIN,MODAL,MAX
2,1. FRUTAS NACIONAIS,,,,,,
3,ABACATE,,Cx 15 kg,"-52,00%","50,00","60,00","65,00"
4,ABACAXI ANANÁS,Grande,"Unid 2,5 kg",S/C,,,


fazendo tratamento na tabela 1 do pdf modelo 2

In [5]:
# Cabeçalho
header = ['PRODUTOS', 'TIPO', 'UNIDADE EMBALAGEM', 'VARIAÇÃO ULTIMOS 12 MESES', 'MIN', 'MODAL', 'MAX']

#c Classes de alimentos
classes = ['1. FRUTAS NACIONAIS',
          '2. FRUTAS IMPORTADAS',
          '3. HORTALIÇAS FRUTO',
          '4. HORTALIÇAS FOLHA, FLOR E HASTE',
          '5. HORTALIÇAS RAIZ, BULBO,TUBÉRCULO E RIZOMA',
          '6. OVOS',
          '7. PEIXE']



##### t

In [ ]:
#####

In [ ]:
# tratamento v2 pag 1 tb 1
def scan_tb_v2_p1(df):
    df.columns = header

    # remove as 2 linhas inicias 
    df = df.iloc[2: , :]

    # adiciona coluna classe
    df['CLASSE'] = ''
    
    # Replace all instances of "" with "0" in columns: 'MIN', 'MODAL', 'MAX'
    df['MIN'] = df['MIN'].str.replace("^$", "0", regex=True)
    df['MODAL'] = df['MODAL'].str.replace("^$", "0", regex=True)
    df['MAX'] = df['MAX'].str.replace("^$", "0", regex=True)


    # Replace all simbols of "" with "S/C" and "%" in columns: 'UNIDADE EMBALAGEM','VARIAÇÃO ULTIMOS 12 MESES'
    df.loc[df['UNIDADE EMBALAGEM'].str.lower() == "".lower(), 'UNIDADE EMBALAGEM'] = "S/C"
    df['VARIAÇÃO ULTIMOS 12 MESES'] = df['VARIAÇÃO ULTIMOS 12 MESES'].str.replace("s/c", "", case=False, regex=False)
    df['VARIAÇÃO ULTIMOS 12 MESES'] = df['VARIAÇÃO ULTIMOS 12 MESES'].str.replace("%", "", case=False, regex=False)
    df['UNIDADE EMBALAGEM'] = df['UNIDADE EMBALAGEM'].str.replace("s/c", "", case=False, regex=False)

    
    # Replace all instances of "," with "." in columns: 'MIN', 'MODAL', 'MAX'
    df['MIN'] = df['MIN'].str.replace(",", ".", case=False, regex=False)
    df['MODAL'] = df['MODAL'].str.replace(",", ".", case=False, regex=False)
    df['MAX'] = df['MAX'].str.replace(",", ".", case=False, regex=False)
    
    # Change column type to float32 for columns: 'MIN', 'MODAL', 'MAX'
    df = df.astype({'MIN': 'float32', 'MODAL': 'float32', 'MAX': 'float32'})
    
    # Change column type to string for columns: 'CLASSE', 'PRODUTOS' and 2 other columns
    df = df.astype({'CLASSE': 'string', 'PRODUTOS': 'string', 'TIPO': 'string', 'UNIDADE EMBALAGEM': 'string'})
    
    # Convert text to uppercase in column: 'TIPO'
    df['TIPO'] = df['TIPO'].str.upper()
    return df

df_scan_tb_v2_p1 = scan_tb_v2_p1( extrair_tabela_pdf(pdf_v2,0) )
df_scan_tb_v2_p1.head()

In [ ]:
# tratamento v2 pag 2 tb 1
def scan_tb_v2_p2(df):
    df.columns = header
    
    df = df.iloc[2: , :]
    
    # classe = df['PRODUTOS'].iloc[0] if str(df['PRODUTOS'].iloc[0]) in classes else ""
    df['CLASSE'] = ''
    
    # Replace all instances of "" with "0" in columns: 'MIN', 'MODAL', 'MAX'
    df['MIN'] = df['MIN'].str.replace("^$", "0", regex=True)
    df['MODAL'] = df['MODAL'].str.replace("^$", "0", regex=True)
    df['MAX'] = df['MAX'].str.replace("^$", "0", regex=True)
    
    # Replace all instances of "," with "." in columns: 'MIN', 'MODAL', 'MAX'
    df['MIN'] = df['MIN'].str.replace(",", ".", case=False, regex=False)
    df['MODAL'] = df['MODAL'].str.replace(",", ".", case=False, regex=False)
    df['MAX'] = df['MAX'].str.replace(",", ".", case=False, regex=False)
    
    # Change column type to float32 for columns: 'MIN', 'MODAL', 'MAX'
    df = df.astype({'MIN': 'float32', 'MODAL': 'float32', 'MAX': 'float32'})

    # Change column type to string for columns: 'CLASSE', 'PRODUTOS' and 2 other columns
    df = df.astype({'CLASSE': 'string', 'PRODUTOS': 'string', 'TIPO': 'string', 'UNIDADE EMBALAGEM': 'string'})
    
    # Convert text to uppercase in column: 'TIPO'
    df['TIPO'] = df['TIPO'].str.upper()

    # Replace all simbols of "" with "S/C" and "%" in columns: 'UNIDADE EMBALAGEM','VARIAÇÃO ULTIMOS 12 MESES'
    df.loc[df['UNIDADE EMBALAGEM'].str.lower() == "".lower(), 'UNIDADE EMBALAGEM'] = "S/C"
    df['VARIAÇÃO ULTIMOS 12 MESES'] = df['VARIAÇÃO ULTIMOS 12 MESES'].str.replace("s/c", "", case=False, regex=False)
    df['VARIAÇÃO ULTIMOS 12 MESES'] = df['VARIAÇÃO ULTIMOS 12 MESES'].str.replace("%", "", case=False, regex=False)
    df['UNIDADE EMBALAGEM'] = df['UNIDADE EMBALAGEM'].str.replace("s/c", "", case=False, regex=False)
    
    return df

df_scan_tb_v2_p2 = scan_tb_v2_p2( extrair_tabela_pdf(pdf_v2,1) )
df_scan_tb_v2_p2.head()


In [ ]:
# tratamento v2 pag 3 tb 1 
def scan_tb_v2_p3(df):
    # define cabeçalhos
    df.columns = header
    
    # Remove primeiras linhas
    df = df.iloc[2: , :]
    
    # Adiciona colula classe
    df['CLASSE'] = ''
    
    # Replace all instances of "" with "0" in columns: 'MIN', 'MODAL', 'MAX'
    df['MIN'] = df['MIN'].str.replace("^$", "0", regex=True)
    df['MODAL'] = df['MODAL'].str.replace("^$", "0", regex=True)
    df['MAX'] = df['MAX'].str.replace("^$", "0", regex=True)
    
    # Replace all simbols of "" with "S/C" and "%" in columns: 'UNIDADE EMBALAGEM','VARIAÇÃO ULTIMOS 12 MESES'
    df.loc[df['UNIDADE EMBALAGEM'].str.lower() == "".lower(), 'UNIDADE EMBALAGEM'] = "S/C"
    df['VARIAÇÃO ULTIMOS 12 MESES'] = df['VARIAÇÃO ULTIMOS 12 MESES'].str.replace("s/c", "", case=False, regex=False)
    df['VARIAÇÃO ULTIMOS 12 MESES'] = df['VARIAÇÃO ULTIMOS 12 MESES'].str.replace("%", "", case=False, regex=False)
    df['UNIDADE EMBALAGEM'] = df['UNIDADE EMBALAGEM'].str.replace("s/c", "", case=False, regex=False)
    
    # Replace all instances of "," with "." in columns: 'MIN', 'MODAL', 'MAX'
    df['MIN'] = df['MIN'].str.replace(",", ".", case=False, regex=False)
    df['MODAL'] = df['MODAL'].str.replace(",", ".", case=False, regex=False)
    df['MAX'] = df['MAX'].str.replace(",", ".", case=False, regex=False)
    
    # Change column type to float32 for columns: 'MIN', 'MODAL', 'MAX'
    df = df.astype({'MIN': 'float32', 'MODAL': 'float32', 'MAX': 'float32'})
    
    # Change column type to string for columns: 'CLASSE', 'PRODUTOS' and 2 other columns
    df = df.astype({'CLASSE': 'string', 'PRODUTOS': 'string', 'TIPO': 'string', 'UNIDADE EMBALAGEM': 'string'})
    
    # Convert text to uppercase in column: 'TIPO'
    df['TIPO'] = df['TIPO'].str.upper()
    return df

df_scan_tb_v2_p3 = scan_tb_v2_p3( extrair_tabela_pdf(pdf_v2,2))
df_scan_tb_v2_p3.head()

##### tramento PDF v1

###### tratamento pdf v1 p1 t1 

In [139]:
df = extrair_tabela_pdf(pdf_v1, 0)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,,Produto – Classe – Tipo,,,Códigos,None,None,None,,,Cotação ( R$ ),None,None,None,None,None,None,
1,,"Folhas, Flores e Hastes",,,Produto,,,Embalagem,,,Mínima,,,Mais Comum,,,Máxima,
2,Acelga (PGM 20 KG-8 CAB),None,None,ACE,None,None,PG 8,None,None,"15,00",None,None,"18,00",None,None,"20,00",None,None
3,"Agrião (MOL 0,25 KG)",None,None,AGR,None,None,M 25,None,None,"0,70",None,None,"0,70",None,None,"1,00",None,None
4,Aipo/Salsão (AMA 13Kg – 6 MOL),None,None,SLS,None,None,A 13,None,None,"12,00",None,None,"15,00",None,None,"15,00",None,None
5,Alcachofra (CXT 2 Kg),None,None,ALC,None,None,C 2,None,None,,None,None,Sem cotação,None,None,,None,None
6,"Alecrim (MOL 0,15 Kg)",None,None,ALE,None,None,M 15,None,None,"1,50",None,None,"2,00",None,None,"3,00",None,None
7,,ALFACE CRESPA (PGM 6 Kg – 18UN,,,,,,,,,,,,,,,,
8,*Extra,None,None,ALFCA,None,None,E 6,None,None,"15,00",None,None,"18,00",None,None,"20,00",None,None
9,*Especial,None,None,ALFCB,None,None,E 6,None,None,"10,00",None,None,"15,00",None,None,"18,00",None,None


In [138]:
# Cabeçalho
header = ['PRODUTOS', 'TIPO', 'UNIDADE EMBALAGEM', 'VARIAÇÃO ULTIMOS 12 MESES', 'MIN', 'MODAL', 'MAX']

#c Classes de alimentos
classes = ['1. FRUTAS NACIONAIS',
          '2. FRUTAS IMPORTADAS',
          '3. HORTALIÇAS FRUTO',
          '4. HORTALIÇAS FOLHA, FLOR E HASTE',
          '5. HORTALIÇAS RAIZ, BULBO,TUBÉRCULO E RIZOMA',
          '6. OVOS',
          '7. PEIXE']

# tratamendo geral pdf v2
def tratamento_1_tbv1(df):
    # define cabeçalhos
    df.columns = header
    
    # Remove primeiras linhas
    df = df.iloc[2: , :]
    
    # Adiciona colula classe
    df.loc[:, 'CLASSE'] = ''

    return df

# Compreensão de lista para criar a lista de dataframes
# list_df_tratamento_1_tbv2 = [ tratamento_1_tbv1( extrair_tabela_pdf(pdf_v1, i)) for i in range(14) ]

# Concatenar os dataframes da lista
# df = pd.concat(list_df_tratamento_1_tbv2.copy())

ValueError: Length mismatch: Expected axis has 18 elements, new values have 7 elements

##### tramento PDF v2

In [35]:
import pandas as pd

# Cabeçalho
header = ['PRODUTOS', 'TIPO', 'UNIDADE EMBALAGEM', 'VARIAÇÃO ULTIMOS 12 MESES', 'MIN', 'MODAL', 'MAX']

#c Classes de alimentos
classes = ['1. FRUTAS NACIONAIS',
          '2. FRUTAS IMPORTADAS',
          '3. HORTALIÇAS FRUTO',
          '4. HORTALIÇAS FOLHA, FLOR E HASTE',
          '5. HORTALIÇAS RAIZ, BULBO,TUBÉRCULO E RIZOMA',
          '6. OVOS',
          '7. PEIXE']

# tratamendo geral pdf v2
def tratamento_1_tbv2(df):
    # define cabeçalhos
    df.columns = header
    
    # Remove primeiras linhas
    df = df.iloc[2: , :]
    
    # Adiciona colula classe
    df.loc[:, 'CLASSE'] = ''

    return df

def tratamento_2_tbv2(df, arquivo='', data_pdf = ''):
    # preencher class
    df.reset_index(drop=True, inplace=True)

    # Colulas de arquivo e data
    df['DATA'] = data_pdf
    df['ARQUIVO'] = arquivo
    
    # preenche classe
    def preenche_classe(df):
        df = df.reset_index(drop=True)    
        
        current_class = ''  # Para rastrear a classe atual
        for index, row in df.iterrows():
            if row['PRODUTOS'] in classes:
                current_class = row['PRODUTOS']
            df.at[index,'CLASSE'] = current_class
    
        df = df[df['PRODUTOS'] != df['CLASSE']]
        df = df.reset_index(drop=True)    
    
        return df
    
    df = preenche_classe(df)
    # Replace all instances of "" with "0" in columns: 'MIN', 'MODAL', 'MAX'
    df['MIN'] = df['MIN'].str.replace("^$", "0", regex=True)
    df['MODAL'] = df['MODAL'].str.replace("^$", "0", regex=True)
    df['MAX'] = df['MAX'].str.replace("^$", "0", regex=True)
    
    # Replace all instances of "," with "." in columns: 'MIN', 'MODAL', 'MAX'
    df['MIN'] = df['MIN'].str.replace(",", ".", case=False, regex=False)
    df['MODAL'] = df['MODAL'].str.replace(",", ".", case=False, regex=False)
    df['MAX'] = df['MAX'].str.replace(",", ".", case=False, regex=False)
    
    # Change column type to float32 for columns: 'MIN', 'MODAL', 'MAX'
    df = df.astype({'MIN': 'float32', 'MODAL': 'float32', 'MAX': 'float32'})
    
    # Change column type to string for columns: 'CLASSE', 'PRODUTOS' and 2 other columns
    df = df.astype({'CLASSE': 'string', 'PRODUTOS': 'string', 'TIPO': 'string', 'UNIDADE EMBALAGEM': 'string'})
    
    # Convert text to uppercase in column: 'TIPO'
    df['TIPO'] = df['TIPO'].str.upper()
    
    # Replace all simbols of "" with "S/C" and "%" in columns: 'UNIDADE EMBALAGEM','VARIAÇÃO ULTIMOS 12 MESES'
    df.loc[df['UNIDADE EMBALAGEM'].str.lower() == "".lower(), 'UNIDADE EMBALAGEM'] = "S/C"
    df['VARIAÇÃO ULTIMOS 12 MESES'] = df['VARIAÇÃO ULTIMOS 12 MESES'].str.replace("s/c", "", case=False, regex=False)
    df['VARIAÇÃO ULTIMOS 12 MESES'] = df['VARIAÇÃO ULTIMOS 12 MESES'].str.replace("%", "", case=False, regex=False)
    df['UNIDADE EMBALAGEM'] = df['UNIDADE EMBALAGEM'].str.replace("s/c", "", case=False, regex=False)
    return df


def pdf_v2_para_tabela(pdf_v2,data=None,):        
    # Compreensão de lista para criar a lista de dataframes
    list_df_tratamento_1_tbv2 = [ tratamento_1_tbv2( extrair_tabela_pdf(PASTA_PDFs + pdf_v2, i)) for i in range(5) ]

    # Concatenar os dataframes da lista
    df = pd.concat(list_df_tratamento_1_tbv2.copy())
    return tratamento_2_tbv2(df,pdf_v2,data)


# df = pdf_v2_para_tabela(PASTA_PDFs +'CEASA-RJ_Boletim_diário_de_preços__09_11_2023.pdf')
# df

In [15]:
# # """
# # Cell generated by Data Wrangler.
# # """
# def tratamento_2_tbv2(df):
#     # preencher class
#     df.reset_index(drop=True, inplace=True)
    
#     # preenche classe
#     def preenche_classe(df):
#         df = df.reset_index(drop=True)    
        
#         current_class = ''  # Para rastrear a classe atual
#         for index, row in df.iterrows():
#             if row['PRODUTOS'] in classes:
#                 current_class = row['PRODUTOS']
#             df.at[index,'CLASSE'] = current_class
    
#         df = df[df['PRODUTOS'] != df['CLASSE']]
#         df = df.reset_index(drop=True)    
    
#         return df
    
#     df = preenche_classe(df)
#     # Replace all instances of "" with "0" in columns: 'MIN', 'MODAL', 'MAX'
#     df['MIN'] = df['MIN'].str.replace("^$", "0", regex=True)
#     df['MODAL'] = df['MODAL'].str.replace("^$", "0", regex=True)
#     df['MAX'] = df['MAX'].str.replace("^$", "0", regex=True)
    
#     # Replace all instances of "," with "." in columns: 'MIN', 'MODAL', 'MAX'
#     df['MIN'] = df['MIN'].str.replace(",", ".", case=False, regex=False)
#     df['MODAL'] = df['MODAL'].str.replace(",", ".", case=False, regex=False)
#     df['MAX'] = df['MAX'].str.replace(",", ".", case=False, regex=False)
    
#     # Change column type to float32 for columns: 'MIN', 'MODAL', 'MAX'
#     df = df.astype({'MIN': 'float32', 'MODAL': 'float32', 'MAX': 'float32'})
    
#     # Change column type to string for columns: 'CLASSE', 'PRODUTOS' and 2 other columns
#     df = df.astype({'CLASSE': 'string', 'PRODUTOS': 'string', 'TIPO': 'string', 'UNIDADE EMBALAGEM': 'string'})
    
#     # Convert text to uppercase in column: 'TIPO'
#     df['TIPO'] = df['TIPO'].str.upper()
    
#     # Replace all simbols of "" with "S/C" and "%" in columns: 'UNIDADE EMBALAGEM','VARIAÇÃO ULTIMOS 12 MESES'
#     df.loc[df['UNIDADE EMBALAGEM'].str.lower() == "".lower(), 'UNIDADE EMBALAGEM'] = "S/C"
#     df['VARIAÇÃO ULTIMOS 12 MESES'] = df['VARIAÇÃO ULTIMOS 12 MESES'].str.replace("s/c", "", case=False, regex=False)
#     df['VARIAÇÃO ULTIMOS 12 MESES'] = df['VARIAÇÃO ULTIMOS 12 MESES'].str.replace("%", "", case=False, regex=False)
#     df['UNIDADE EMBALAGEM'] = df['UNIDADE EMBALAGEM'].str.replace("s/c", "", case=False, regex=False)
#     return df


# df_tratamento_2_tbv2 = tratamento_2_tbv2(df.copy())
# df_tratamento_2_tbv2.head()



In [50]:
from datetime import datetime as dt
# Suprime os avisos SettingWithCopyWarning
pd.options.mode.chained_assignment = None


def extrair_tb_pdf_V2():
    # pass

    df_pdfs_v2 = pd.read_csv("./dados/ceasa_lista_pdf.csv", sep=',')

    df_pdfs_v2 = df_pdfs_v2.drop_duplicates(subset=['NOME_ARQUIVO'])
    df_pdfs_v2 = df_pdfs_v2[:160]
    # df_pdfs_v2 =  df_pdfs_v2.loc[df['NOME_ARQUIVO'] == 'CEASA-RJ_Boletim_diário_de_preços__22_05_2023.pdf']
    # df_pdfs_v2 = pd.DataFrame({"index": 0, "DATA": "01-01-2024",
    #                           "NOME_ARQUIVO": "CEASA-RJ_Boletim_diário_de_preços__22_05_2023.pdf"})

    len = df_pdfs_v2.shape[0]
    lista_pdf_v2 = []
    for index, row in df_pdfs_v2.iterrows():
        arquivo = row['NOME_ARQUIVO']
        data = row['DATA']
        print_percent_done(index, len, data)

        data = dt.strptime(data, "%d-%m-%Y")
        data_pdf_v2 = dt.strptime('01-03-2023', "%d-%m-%Y")

        if data > data_pdf_v2:
            df = pdf_v2_para_tabela(
                arquivo, data.strftime("%d-%m-%Y"))

            lista_pdf_v2.append(df)

    df = pd.concat(lista_pdf_v2)
    return df


df = extrair_tb_pdf_V2()
# df = df.astype('float32', errors='ignore').to_feather('tmp/historical-raw')


half_floats = df.select_dtypes(include="float16")
df[half_floats.columns] = half_floats.astype("float32")
df.to_feather('tmp/historical-raw')

df.to_csv('dados_pdf_v2.csv')
df.to_parquet('myfile.parquet')

# df.to
# df.to_parquet('dados_pdf_v2.parquet.gzip',
#               compression='gzip')

ValueError: could not convert string to float: '6..00': Error while type casting for column 'MIN'